In [1]:
import pennylane as qml
from pennylane import numpy as np

In [7]:
from problem_3_data import *

In [26]:
import pennylane as qml
from pennylane import numpy as np
from fractions import Fraction


def fractional_binary_to_float(sample):
    return np.sum(
        [int(sample[bit]) / 2 ** (bit + 1) for bit in range(len(sample))]
    )



def phase_to_order(phase, max_denominator):
    s_over_r = Fraction(phase)
    return s_over_r.limit_denominator(max_denominator).denominator


def get_U_Na(N, a):
    n_qubits = int(np.ceil(np.log2(N)))

    U_Na = np.zeros([2 ** n_qubits, 2 ** n_qubits])

    for k in range(N):
        U_Na[(k * a) % N, k] = 1

    for extra in range(N, 2 ** n_qubits):
        U_Na[extra, extra] = 1

    return U_Na

def run_order_finding(a, N):
    U_Na = get_U_Na(a, N)

    num_estimation_qubits = 10
    num_target_qubits = int(np.log2(len(U_Na)))

    estimation_wires = range(num_estimation_qubits)
    target_wires = range(num_estimation_qubits, num_estimation_qubits + num_target_qubits)

    dev = qml.device('default.qubit', wires=num_estimation_qubits+num_target_qubits, shots=1)

    @qml.qnode(dev)
    def find_order():
        # Prepare target register
        qml.PauliX(wires=target_wires[-1])

        # Do phase estimation
        qml.QuantumPhaseEstimation(
            U_Na,
            estimation_wires=estimation_wires,
            target_wires=target_wires
        )

        return qml.sample(wires=estimation_wires)

    possible_r = []

    for _ in range(10):
        sample = find_order()
        #print(f"Sample = {sample}")
        phase = fractional_binary_to_float(sample)
        #print(f"Numerical phase = {phase}")
        est_r = phase_to_order(phase, N)
        #print(f"Guess for r = {est_r}")
        possible_r.append(est_r)

    return max(possible_r)

def factor_with_shor(N):
    """Implements Shor's algorithm to determine the prime factors of a provided
    value N.

    This function _must_ execute a QNode in order to factor the number.

    Args:
        N (int): The number to factor.

    Returns:
        int, int: p, q such that N = p * q.
    """
    p, q = 0, 0

    def shors_algorithm(N):
        for _ in range(10):
            a = np.random.choice(list(range(2, N-1)))

            r = run_order_finding(a, N)

            if r % 2 == 1:
                continue

            x = (a ** (r // 2)) % N

            if x == 1 or x == (N - 1):
                continue

            p = np.gcd(x - 1, N)
            q = np.gcd(x + 1, N)
            return p, q

    for _ in range(300):
        p, q = shors_algorithm(N)
        if p * q == N:
            print(f"p={p}\nq={q}")
            break

    return p, q


def decode_message(message, key_pair):
    """Use Shor's algorithm to decrypt an arbitrary secret message encoded using
    the provided RSA public key pair.

    Messages are encoded as a list of integers; the mapping between characters in
    the message and `decoded` integers is:
      - 0-9: numbers 0-9
      - 10-35: letters a-z (only lowercase is used)
      - 36: space

    Args:
        message (List[int]): A list of integers representing the secret message.
            Each integer in the list represents a different character in the  message.
        key_pair ((int, int)): The public RSA key (e, N).

    Returns:
        str: The decoded message.
    """
    decoded_message = ""

    def get_d(e, theta):
        for d in range(1, theta):
            if (d * e) % N == 1:
                # print(f"D= {d} for e= {e}")
                return d

    p = 13
    q = 23

    theta = (p-1) * (q-1)
    e, N = key_pair
    d = get_d(e, theta)

    letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

    def int_to_char(num):
        if num <= 9:
            return str(num)
        if 10 <= num <= 35:
            index = num - 10
            return letters[index]
        if num == 36:
            return ' '

    for msg in message:
        decoded_int = msg ** d % N % 37
        decoded_message += int_to_char(decoded_int)


    return decoded_message




def find_party_location(party_message, party_key_pair):
    """Recover the location of the surprise party.

    *Note that there is no explicit test function provided for this; a hidden
    test function will be used (with just this one test case), and is worth 1
    point. The values that will be passed to the test function are in
    problem_3_data.py.*

    Args:
        party_message (List[int]): A list of integers representing the secret message.
            Each integer in the list represents a different character in the message.
        party_key_pair ((int, int)): The public RSA key (e, N).

    Returns:
        str: a 10-character string indicating the location of the surprise party.

    """

    location = ""

    e, N = party_key_pair

    p, q  = factor_with_shor(N)

    decode_message(party_message, party_key_pair)


    return location

decode_message(party_message, party_key_pair)

'22kbvkbq1vibovbvib7k2k7qbmviobbvbvoq2qvvkb2bqm7bbmbx21bbmbrvmvb7kb7m'

In [6]:
decode_message(party_message, party_key_pair)

263

In [11]:
e = 25
N = 299
def get_d(e, theta):
    for d in range(1, theta):
        if (d * e) % N == 1:
            print(f"D= {d} for e= {e}")
            return d

D= 12 for e= 25
